In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
## Sklearn package
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor 
from sklearn.preprocessing import RobustScaler, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import mean_squared_error 
from scipy.stats import skew, kurtosis
import warnings
warnings.filterwarnings("ignore")
pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 99

In [3]:
df = pd.read_csv("data/raw/train_logs.csv")

In [4]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=123)

In [5]:
train_df.head()

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
1733243,363cff31,1833,692402,692566,164,Input,q,q,q,1284,247
4485593,8bcd94be,1122,536640,536724,84,Input,q,q,q,686,113
87335,0249a095,4814,1781257,1781289,32,Nonproduction,ArrowRight,ArrowRight,NoChange,2518,500
6792139,d1a0c0dc,1228,1812140,1812235,95,Input,q,q,q,747,117
2825825,582dbb0e,731,596397,596456,59,Input,q,q,q,439,72


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6724718 entries, 1733243 to 4967934
Data columns (total 11 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   id               object
 1   event_id         int64 
 2   down_time        int64 
 3   up_time          int64 
 4   action_time      int64 
 5   activity         object
 6   down_event       object
 7   up_event         object
 8   text_change      object
 9   cursor_position  int64 
 10  word_count       int64 
dtypes: int64(6), object(5)
memory usage: 615.7+ MB


In [7]:
train_df.describe()

,event_id,down_time,up_time,action_time,cursor_position,word_count
count,6.724718e+06,6.724718e+06,6.724718e+06,6.724718e+06,6.724718e+06,6.724718e+06
mean,2.067431e+03,7.934946e+05,7.935927e+05,9.809989e+01,1.222872e+03,2.314426e+02
std,1.588124e+03,5.149524e+05,5.149502e+05,2.422870e+02,9.485225e+02,1.758954e+02
min,1.000000e+00,1.060000e+02,2.520000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.520000e+02,3.731730e+05,3.732670e+05,6.600000e+01,4.990000e+02,9.600000e+01
50%,1.726000e+03,7.207530e+05,7.208520e+05,9.300000e+01,1.043000e+03,2.000000e+02
75%,2.925000e+03,1.162892e+06,1.162992e+06,1.220000e+02,1.706000e+03,3.270000e+02
max,1.287600e+04,8.313630e+06,8.313707e+06,4.474700e+05,7.802000e+03,1.326000e+03


In [8]:
train_df.isnull().sum()

id                 0
event_id           0
down_time          0
up_time            0
action_time        0
activity           0
down_event         0
up_event           0
text_change        0
cursor_position    0
word_count         0
dtype: int64